First, import needed modules

In [38]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt

Initalize variables

In [39]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Setup figs_tex

In [40]:
# Setup TeX Code to integrate figures
fig_tex = r'(\renewcommand{\thefigure}{%s} \begin{figure} \centering \includegraphics[width=0.9\textwidth]{Figures/%s} \captionlistentry{%s} \label{fig:%s} \end{figure} \clearpage)'
figs_tex = {}

Fig E1: Municipalities affected by each treatment

In [41]:
pl_05 = gpd.read_file(f'{path}/data/geodata/powerlines_05.shp')
pl_09 = gpd.read_file(f'{path}/data/geodata/new_powerlines_09.shp')
pl_13 = gpd.read_file(f'{path}/data/geodata/new_powerlines_13.shp')
all_powerlines = gpd.read_file(f'{path}/data/geodata/all_powerlines.shp')
municipalities = gpd.read_file(f'{path}/data/geodata/municipalities.shp')
mun_unincorporated = gpd.read_file(f'{path}/data/geodata/unincorporated.shp')
states = gpd.read_file(f'{path}/data/geodata/VG250_LAN.shp')

In [42]:
legend_dict = {2005: 'DENA 2005', 2009: 'EnLAG 2009', 2013: 'BBPlG 2013'}
tvars = {2005: 't2005', 2009: 't2009', 2013: 't2013'}
all_powerlines['year'] = all_powerlines['year'].astype(int)
# initialize figure 
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 6), dpi=600, constrained_layout=True)
# title for entire figure
fig.suptitle('Fig. E1: \n Municipalities affected by Powerline Projects', fontsize=16)
for i, (year, desc) in enumerate(legend_dict.items()):
    # subset powerlines
    plot_pl = all_powerlines[all_powerlines['year'] == year]
    # subset treated muns
    treated_muns = municipalities[municipalities[tvars[year]] != 'None']
    # draw subfig
    ax[i].set_aspect('equal')
    ax[i].set_adjustable('datalim', share=True)
    ax[i].set_axis_off()
    ax[i].set_title(f'\n {desc}')
    municipalities.plot(ax=ax[i], color='#4f6393', zorder=1)
    if year == 2005:
        treated_muns.plot(ax=ax[i], column='t2005', cmap='viridis', zorder=2)
    if year == 2009:
        treated_muns.plot(ax=ax[i], column='t2009', cmap='viridis', legend=True, zorder=2, 
        legend_kwds={'loc':'lower center', 'ncol':2, 'title':'Treatments'})
    if year == 2013:
        treated_muns.plot(ax=ax[i], column='t2013', cmap='viridis', zorder=2)
    states.boundary.plot(ax=ax[i], color='white', lw = 0.05, zorder=3)
    plot_pl.plot(ax=ax[i], color='red', lw=2, zorder=4)
plt.savefig(f'{path}/figures/py/E1_all_treatments.png', bbox_inches="tight", pad_inches=0)
plt.close()

figs_tex[1]=(fig_tex % ('E1', 'E1_all_treatments.png', 'Municipalities affected by Powerline Projects', 'E1'))

Fig E2: Municipalities (already) affected by each year

In [43]:
legend_dict = {2005: 'DENA 2005', 2009: 'EnLAG 2009', 2013: 'BBPlG 2013'}
all_powerlines['year'] = all_powerlines['year'].astype(int)
# initialize figure 
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 6), dpi=600, constrained_layout=True)
# title for entire figure
fig.suptitle('Fig. E2: \n Treated municipalities by year', fontsize=16)
for i, (year, desc) in enumerate(legend_dict.items()):
    # subset powerlines
    plot_pl = all_powerlines[all_powerlines['year'] <= year]     
    # draw subfig
    ax[i].set_aspect('equal')
    ax[i].set_adjustable('datalim', share=True)
    ax[i].set_axis_off()
    ax[i].set_title(f'\n {year}')
    municipalities.plot(ax=ax[i], color='#4f6393', zorder=1)
    if year == 2005:
        # subset treated muns
        treated_muns = municipalities[municipalities['t2005'] != 'None']
        treated_muns.plot(ax=ax[i], column='t2005', cmap='viridis', zorder=2)
    if year == 2009:
        treated_muns = municipalities[municipalities['et2010'] != 'None']
        treated_muns.plot(ax=ax[i], column='et2010', cmap='viridis', legend=True, zorder=2, 
            legend_kwds={'loc':'lower center', 'ncol':2, 'title':'Treatments'})
    if year == 2013:
        treated_muns = municipalities[municipalities['et2014'] != 'None']
        treated_muns.plot(ax=ax[i], column='et2014', cmap='viridis', zorder=2)
    states.boundary.plot(ax=ax[i], color='white', lw = 0.05, zorder=3)
    plot_pl.plot(ax=ax[i], color='red', lw=2, zorder=4)
plt.savefig(f'{path}/figures/py/E2_all_treatments_sum.png', bbox_inches="tight", pad_inches=0)
plt.close()

figs_tex[2]=(fig_tex % ('E2', 'E2_all_treatments_sum.png', 'Treated municipalities by year', 'E2'))

Fig E4: Search Trends

In [44]:
gtrends = pd.read_csv(f'{path}/data/gtrends/stromtrasse.csv')
gtrends['Date'] = pd.to_datetime(gtrends['Monat'])

In [45]:
import matplotlib.dates as mdates
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 3), dpi=600, constrained_layout=True)
fig.suptitle('Fig. E4: \n Google Search interest for the term "Stromtrasse"', fontsize=16)
ax.plot(gtrends['Date'], gtrends['stromtrasse: (Deutschland)']) 
ax.xaxis.set_major_locator(mdates.YearLocator(1))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
ax.set_xlim(right=pd.Timestamp("2022-08-30 00:00:00"), left=pd.Timestamp("2004-01-01 00:00:00"))
ax.set_ylim(-1, 101)
ax.set_ylabel('Search Interest', fontsize=10)
plt.setp(ax.get_xticklabels(), rotation=30, ha="right")
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.savefig(f'{path}/figures/py/E4_gtrends.png')
plt.close()

figs_tex[4]=(fig_tex % ('E4', 'E4_gtrends.png', 'Google Search interest for the term "Stromtrasse"', 'E4'))

First, import needed modules

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

Initalize variables

In [47]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [48]:
btw = pd.read_csv(f'{path}/data/btw_treat.csv', encoding = 'UTF-8-SIG', sep=',', converters={'AGS': str})

View summary statistics

In [49]:
columns = ['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
btw.describe().to_csv(f'{path}/tables/btw_descriptives.csv', sep=';', encoding='utf-8-sig', decimal=',', columns=columns, float_format='%.2f')
btw[columns].describe()

,Union,SPD,FDP,Linke,Grüne,Andere
count,78110.000000,78110.000000,78110.000000,78110.000000,78110.000000,78110.000000
mean,43.988886,29.875659,6.749605,7.121998,5.653732,6.610120
std,12.010638,12.288360,3.997007,8.168828,3.191483,6.491112
min,1.098432,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.319335,20.143136,3.809024,0.836478,3.320797,2.298363
50%,42.916331,29.077540,5.985883,4.137995,5.107530,4.559065
75%,51.644096,38.519130,8.703088,9.949111,7.325108,8.523961
max,100.000000,85.294118,52.373418,55.294118,50.000000,62.857143


In [50]:
btw['year'].value_counts().to_csv(f'{path}/tables/btw_obsyears.csv', sep=';', encoding='utf-8-sig', decimal=',', float_format='%.2f')

Fig E3: Plot general development of first and second vote shares

In [51]:
btw_eszs = pd.read_csv(f'{path}/data/btw_eszs.csv', encoding = 'utf-8-sig')
btw_eszs['vote_type'] = np.where(btw_eszs['first_vote'] == 1, 'First Vote', 'Second Vote')

In [52]:
avg_eszs_vote_shares = btw_eszs[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'year', 'vote_type']].groupby(['year', 'vote_type'], as_index = False).mean()
avg_both_vote_shares = btw_eszs[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'year']].groupby(['year'], as_index = False).mean()
avg_both_vote_shares['vote_type'] = 'Mean'
avg_vote_shares = pd.concat([avg_eszs_vote_shares, avg_both_vote_shares], sort=True)
avg_vote_shares.head()

,Andere,FDP,Grüne,Linke,SPD,Union,vote_type,year
0,2.612058,3.376459,4.965478,3.657808,34.555566,50.832633,First Vote,1994
1,3.518688,6.234391,5.464208,3.979173,33.479900,47.323640,Second Vote,1994
2,3.635418,3.156431,3.807150,4.884896,40.366090,44.150015,First Vote,1998
3,6.183717,5.827799,4.954207,5.153792,38.169901,39.710585,Second Vote,1998
4,1.065732,5.902271,4.181176,4.434954,38.008248,46.407619,First Vote,2002


In [53]:
blacks = ['#000000', '#191919', 'blue']
reds = ['#cc0000', '#d11919', 'blue']
yellows = ['#ffd075', '#ffc75c', 'blue']
purples = ['#800080', '#8c198c', 'blue']
greens = ['#008000', '#198c19', 'blue']
greys = ['#808080', '#8c8c8c', 'blue']
markers = ['o', 's', 'h']
vote_types = ['First Vote', 'Second Vote', 'Mean']

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 10), dpi=600, layout="tight")
#set x labels
plt.setp(axes, xticks=[1994,1998,2002,2005,2009,2013,2017])
# set legend font size
plt.rc('legend',fontsize=6)
# title for entire figure
fig.suptitle('Fig. E3: Average First and Second Vote Shares in Federal Elections', fontsize=16)

for i, type in enumerate(vote_types):
    if type == 'Mean':
        ls = '--'
    else:
        ls = '-'
    plot_data = avg_vote_shares[avg_vote_shares['vote_type'] == type]
    # edit subplots
    axes[0, 0].set_title('Union', fontsize=12)
    axes[0, 0].set_ylabel('% Votes', fontsize=8)
    axes[0, 0].plot(plot_data['year'], plot_data['Union'], c=blacks[i], marker = markers[i], label=type, lw=0.8, ms=4, ls=ls)
    axes[0, 0].legend(loc='lower left')

    axes[0, 1].set_title('SPD', fontsize=12)
    axes[0, 1].set_ylabel('% Votes', fontsize=8)
    axes[0, 1].plot(plot_data['year'], plot_data['SPD'], c=reds[i], marker = markers[i], label=type, lw=0.8, ms=4, ls=ls)
    axes[0, 1].legend(loc='lower left')

    axes[1, 0].set_title('FDP', fontsize=12)
    axes[1, 0].set_ylabel('% Votes', fontsize=8)
    axes[1, 0].plot(plot_data['year'], plot_data['FDP'], c=yellows[i], marker = markers[i], label=type, lw=0.8, ms=4, ls=ls)
    axes[1, 0].legend(loc='upper left')

    axes[1, 1].set_title('Linke', fontsize=12)
    axes[1, 1].set_ylabel('% Votes', fontsize=8)
    axes[1, 1].plot(plot_data['year'], plot_data['Linke'], c=purples[i], marker = markers[i], label=type, lw=0.8, ms=4, ls=ls)
    axes[1, 1].legend(loc='upper left')

    axes[2, 0].set_title('\n Grüne', fontsize=12)
    axes[2, 0].set_ylabel('% Votes', fontsize=8)
    axes[2, 0].plot(plot_data['year'], plot_data['Grüne'], c=greens[i], marker = markers[i], label=type, lw=0.8, ms=4, ls=ls)
    axes[2, 0].legend(loc='upper left')

    axes[2, 1].set_title(' \n Andere', fontsize=12)
    axes[2, 1].set_ylabel('% Votes', fontsize=8)
    axes[2, 1].plot(plot_data['year'], plot_data['Andere'], c=greys[i], marker = markers[i], label=type, lw=0.8, ms=4, ls=ls)
    axes[2, 1].legend(loc='upper left')

# Save figure
fig.savefig(f'{path}/figures/py/E3_btw_avg_vote_shares.png')
plt.close()
figs_tex[3]=(fig_tex % ('E3', 'E3_btw_avg_vote_shares.png', 'Average First and Second Vote Shares in Federal Elections', 'E3'))

Fig E5: Vote Shares by Ever-treated

In [54]:
avg_ever_treated = btw[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'ever_treated', 'year']].groupby(['ever_treated', 'year'], as_index = False).mean()
avg_ever_treated.rename(columns = {'ever_treated':'treatment'}, inplace = True)
mean = btw[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'year']].groupby(['year'], as_index = False).mean()
mean['treatment'] = 'Mean'
avg_ever_treated = pd.concat([avg_ever_treated, mean])
avg_ever_treated.head()

,treatment,year,Union,SPD,FDP,Linke,Grüne,Andere
0,Direct Line,1994.0,47.146832,37.108136,5.001208,2.988999,5.278993,2.475832
1,Direct Line,1998.0,40.377097,42.897251,4.411622,4.156877,4.223417,3.933737
2,Direct Line,2002.0,42.462172,40.540018,6.501484,3.646680,5.025751,1.823896
3,Direct Line,2005.0,41.115156,36.099920,7.063406,7.489894,4.762178,3.469446
4,Direct Line,2009.0,39.759503,25.671063,11.865775,10.912602,7.606396,4.184661


In [55]:
treatments = ['None', 'Direct Line', 'Within 0-15 km', 'Within 15-30 km', 'Within 30-50 km', 'Mean']
blacks = ['#000000', '#191919', '#323232', '#4c4c4c', '#666666', 'blue']
reds = ['#cc0000', '#d11919', '#d63232', '#db4c4c', '#e06666', 'blue']
yellows = ['#ffd075', '#ffc75c', '#ffbe42', '#ffb529', '#ffac0f', 'blue']
purples = ['#800080', '#8c198c', '#993299', '#a64ca6', '#b266b2', 'blue']
greens = ['#008000', '#198c19', '#329932', '#4ca64c' , '#66b266', 'blue']
greys = ['#808080', '#8c8c8c', '#999999', '#a6a6a6', '#b2b2b2', 'blue']
markers = ['o', 's', 'v', 'D', '^', 'h']

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 10), dpi=600)
#set x labels
plt.setp(axes, xticks=[1994,1998,2002,2005,2009,2013,2017])
# set legend font size
plt.rc('legend',fontsize=6)
# title for entire figure
fig.suptitle('Fig. E5: \n Average Vote Shares by ever-treated status', fontsize=16)

for i, treatment in enumerate(treatments):
    plot_data = avg_ever_treated[avg_ever_treated['treatment'] == treatment]
    if treatment == 'Mean':
        ls = '--'
    else:
        ls = '-'
    # edit subplots
    axes[0, 0].set_title('Union', fontsize=12)
    axes[0, 0].set_ylabel('% Votes', fontsize=8)
    axes[0, 0].plot(plot_data['year'], plot_data['Union'], c=blacks[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
    axes[0, 0].legend(loc='lower left')

    axes[0, 1].set_title('SPD', fontsize=12)
    axes[0, 1].set_ylabel('% Votes', fontsize=8)
    axes[0, 1].plot(plot_data['year'], plot_data['SPD'], c=reds[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
    axes[0, 1].legend(loc='lower left')

    axes[1, 0].set_title('FDP', fontsize=12)
    axes[1, 0].set_ylabel('% Votes', fontsize=8)
    axes[1, 0].plot(plot_data['year'], plot_data['FDP'], c=yellows[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
    axes[1, 0].legend(loc='upper left')

    axes[1, 1].set_title('Linke', fontsize=12)
    axes[1, 1].set_ylabel('% Votes', fontsize=8)
    axes[1, 1].plot(plot_data['year'], plot_data['Linke'], c=purples[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
    axes[1, 1].legend(loc='upper left')

    axes[2, 0].set_title('Grüne', fontsize=12)
    axes[2, 0].set_ylabel('% Votes', fontsize=8)
    axes[2, 0].plot(plot_data['year'], plot_data['Grüne'], c=greens[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
    axes[2, 0].legend(loc='upper left')

    axes[2, 1].set_title('Andere', fontsize=12)
    axes[2, 1].set_ylabel('% Votes', fontsize=8)
    axes[2, 1].plot(plot_data['year'], plot_data['Andere'], c=greys[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
    axes[2, 1].legend(loc='upper left')

# Save figure
fig.savefig(f'{path}/figures/py/E5_btw_ever_treated_vote_shares.png')
plt.close()
figs_tex[5]=(fig_tex % ('E5', 'E5_btw_ever_treated_vote_shares.png', 'Average Vote Shares by ever-treated status', 'E5'))

Fig E6-E8: Vote shares by treatment status in year

In [56]:
avg_dfs = {2005: 'treatment_05', 2010: 'treatment_09', 2014: 'treatment_13'}
for year, treatment in avg_dfs.items():
    avg_dfs[year] = btw[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', treatment, 'year']].groupby([treatment, 'year'], as_index = False).mean()
    avg_dfs[year].rename(columns = {f'{treatment}':'treatment'}, inplace = True)
    mean = btw[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'year']].groupby(['year'], as_index = False).mean()
    mean['treatment'] = 'Mean'
    avg_dfs[year] = pd.concat([avg_dfs[year], mean])

In [57]:
treatments = ['None', 'Direct Line', 'Within 0-15 km', 'Within 15-30 km', 'Within 30-50 km', 'Mean']
blacks = ['#000000', '#191919', '#323232', '#4c4c4c', '#666666', 'blue']
reds = ['#cc0000', '#d11919', '#d63232', '#db4c4c', '#e06666', 'blue']
yellows = ['#ffd075', '#ffc75c', '#ffbe42', '#ffb529', '#ffac0f', 'blue']
purples = ['#800080', '#8c198c', '#993299', '#a64ca6', '#b266b2', 'blue']
greens = ['#008000', '#198c19', '#329932', '#4ca64c' , '#66b266', 'blue']
greys = ['#808080', '#8c8c8c', '#999999', '#a6a6a6', '#b2b2b2', 'blue']
markers = ['o', 's', 'v', 'D', '^', 'h']

n = 6
for year, df in avg_dfs.items():
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 10), dpi=600)
    #set x labels
    plt.setp(axes, xticks=[1994,1998,2002,2005,2009,2013,2017])
    # set legend font size
    plt.rc('legend',fontsize=6)
    # title for entire figure
    fig.suptitle(f'Fig. E{n}: \n Average Vote Shares by treatment status in {year}', fontsize=16)

    for i, treatment in enumerate(treatments):
        plot_data = df[df['treatment'] == treatment]
        # set ls
        if treatment == 'Mean':
            ls = '--'
        else:
            ls = '-'
        # edit subplots
        axes[0, 0].set_title('Union', fontsize=12)
        axes[0, 0].set_ylabel('% Votes', fontsize=8)
        axes[0, 0].plot(plot_data['year'], plot_data['Union'], c=blacks[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
        axes[0, 0].legend(loc='lower left')
        axes[0, 0].axvline(x=year, c='red', ls='--')

        axes[0, 1].set_title('SPD', fontsize=12)
        axes[0, 1].set_ylabel('% Votes', fontsize=8)
        axes[0, 1].plot(plot_data['year'], plot_data['SPD'], c=reds[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
        axes[0, 1].legend(loc='lower left')
        axes[0, 1].axvline(x=year, c='red', ls='--')

        axes[1, 0].set_title('FDP', fontsize=12)
        axes[1, 0].set_ylabel('% Votes', fontsize=8)
        axes[1, 0].plot(plot_data['year'], plot_data['FDP'], c=yellows[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
        axes[1, 0].legend(loc='upper left')
        axes[1, 0].axvline(x=year, c='red', ls='--')

        axes[1, 1].set_title('Linke', fontsize=12)
        axes[1, 1].set_ylabel('% Votes', fontsize=8)
        axes[1, 1].plot(plot_data['year'], plot_data['Linke'], c=purples[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
        axes[1, 1].legend(loc='upper left')
        axes[1, 1].axvline(x=year, c='red', ls='--')

        axes[2, 0].set_title('Grüne', fontsize=12)
        axes[2, 0].set_ylabel('% Votes', fontsize=8)
        axes[2, 0].plot(plot_data['year'], plot_data['Grüne'], c=greens[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
        axes[2, 0].legend(loc='upper left')
        axes[2, 0].axvline(x=year, c='red', ls='--')

        axes[2, 1].set_title('Andere', fontsize=12)
        axes[2, 1].set_ylabel('% Votes', fontsize=8)
        axes[2, 1].plot(plot_data['year'], plot_data['Andere'], c=greys[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
        axes[2, 1].legend(loc='upper left')
        axes[2, 1].axvline(x=year, c='red', ls='--')

    # Save figure
    fig.savefig(f'{path}/figures/py/E{n}_btw_{year}_vote_shares.png')
    plt.close()
    figs_tex[n] = (fig_tex % (f'E{n}', f'E{n}_btw_{year}_vote_shares.png', f'Average Vote Shares by treatment status in {year}', f'E{n}'))
    n = n + 1

Fig E9: BTW Election winners

In [58]:
# Load shapefiles
import geopandas as gpd
municipalities = gpd.read_file(f'{path}/data/geodata/municipalities.shp')
uninc = gpd.read_file(f'{path}/data/geodata/unincorporated.shp')
states = gpd.read_file(f'{path}/data/geodata/VG250_LAN.shp')
powerlines = gpd.read_file(f'{path}/data/geodata/all_powerlines.shp')
powerlines = powerlines.set_geometry('geometry')
powerlines['year'] = powerlines['year'].astype(int)
# set year to treatment year
powerlines['year'] = np.where(powerlines['year'] == 2013, 2014, powerlines['year'])
powerlines['year'] = np.where(powerlines['year'] == 2009, 2010, powerlines['year'])

In [59]:
# Prepare df with election winners
btw_win = btw[['AGS', 'year', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']]
btw_win['winner'] = btw_win[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']].idxmax(axis = 1)
#btw_win = btw_win[['AGS', 'year', 'winner']]
btw_win = btw_win[btw_win['year'] > 1998]
btw_win['winner'].describe()

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_85109/2197327854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btw_win['winner'] = btw_win[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']].idxmax(axis = 1)


count     55790
unique        6
top       Union
freq      42093
Name: winner, dtype: object

In [60]:
# Merge win and municipalities
btw_win_plot = pd.merge(btw_win, municipalities, on='AGS')
btw_win_plot = btw_win_plot.set_geometry('geometry')

In [61]:
# add color column to df
years = btw_win_plot.year.unique().astype(int).tolist()
cs = {'Union': '#000000', 'SPD': '#cc0000', 'FDP': '#ffff00', 'Grüne': '#008000', 'Linke': '#800080', 'Andere': '#808080'}
btw_win_plot['c'] = 0
for party, c in cs.items():
    btw_win_plot['c'] = np.where(btw_win['winner'] == party, c, btw_win_plot['c'])

In [62]:
from geopandas.tools import overlay
from matplotlib.patches import Circle, Wedge, Polygon
from matplotlib.collections import PatchCollection

fig = plt.figure(layout="constrained", figsize=(11, 6), dpi=600)
subfigs = fig.subfigures(2, 1, height_ratios=[7, 1])
axs0 = subfigs[0].subplots(1, 5)
axs1 = subfigs[1].subplots(1, 2)
for ax1 in axs1:
    ax1.set_aspect('equal')
    ax1.set_axis_off()
# title for entire figure
fig.suptitle(f'Fig. E9 \n Federal Elections - Partys who receiced the largest first vote share', fontsize=16)
for i, year in enumerate(years):
    # subset data
    plot_data = btw_win_plot[btw_win_plot['year'] == year]
    # subset powerlines
    plot_pl = powerlines[powerlines['year'] <= year]
    # setup figure
    axs0[i].set_aspect('equal')
    axs0[i].set_axis_off()
    axs0[i].set_title(f'{year}')
    municipalities.plot(ax=axs0[i], color='lightblue', zorder=1)
    plot_data.plot(ax=axs0[i], color=plot_data['c'], lw=0.01, zorder=2)
    states.boundary.plot(ax=axs0[i], color='white', lw = 0.05, zorder=3)
    plot_pl.plot(ax=axs0[i], color=plot_pl['color'], lw=2, zorder=4)
    # Add legends    
    # add handles for the legends
    p_handles = [
        Circle([0], [0], color='#000000'),
        Circle([0], [0], color='#cc0000'),
        Circle([0], [0], color='#ffff00'),
        Circle([0], [0], color='#800080'),
        Circle([0], [0], color='#008000'),
        Circle([0], [0], color='#808080')
    ]
    pl_handles = [
        Circle([0], [0], color='#1ED4CE'),
        Circle([0], [0], color='#B8FDE7'),
        Circle([0], [0], color='#1B5EB6')
    ]
    # define labels for the legend
    p_labels = ['CDU', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
    pl_labels = ['DENA 2005', 'EnLAG 2009', 'BBPlG 2013']
    # plot legends
    axs1[0].legend(handles=pl_handles, labels=pl_labels, loc='upper center', title='Powerlines', ncol=3)
    axs1[1].legend(handles=p_handles, labels=p_labels, loc='upper center', title='Parties', ncol=6)
    
plt.savefig(f'{path}/figures/py/E9_btw_winners.png')
plt.close()
figs_tex[9] = (fig_tex % (f'E9', f'E9_btw_winners.png', 'Federal Elections - Partys who receiced the largest first vote share', f'E{9}'))

/opt/homebrew/Caskroom/miniconda/base/envs/vscode/lib/python3.10/site-packages/geopandas/plotting.py:693: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(


First, import needed modules

In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

Initalize variables

In [64]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [65]:
ltw = pd.read_csv(f'{path}/data/ltw_treat.csv', encoding = 'UTF-8-SIG', sep=',', converters={'AGS': str})

View summary statistics

In [66]:
columns = ['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
ltw.describe().to_csv(f'{path}/tables/ltw_descriptives.csv', sep=';', encoding='utf-8-sig', decimal=',', columns=columns, float_format='%.2f')
ltw[columns].describe()

,Union,SPD,FDP,Linke,Grüne,Andere
count,14319.000000,14316.000000,14259.000000,14311.000000,14280.000000,14319.000000
mean,41.472536,19.724475,6.319968,7.406899,8.325754,16.856586
std,11.183494,11.567003,3.828827,9.121070,4.967469,10.588800
min,5.797101,0.000000,0.000000,0.202429,0.000000,0.000000
25%,33.668402,10.117269,3.450118,2.198791,4.625162,7.847499
50%,40.861027,17.197551,5.677155,3.601695,7.245246,15.619877
75%,48.653924,28.269766,8.216152,6.122824,10.942920,24.315730
max,95.454545,58.673469,55.555556,47.500000,38.003901,71.186441


In [67]:
ltw[['year', 'Land']].value_counts().to_csv(f'{path}/tables/ltw_obsyears.csv', sep=';', encoding='utf-8-sig', decimal=',', float_format='%.2f')

Fig E10-E17: LTW vote shares by treatment year

In [68]:
avg_dfs = {2014: 'treatment_13', 2010: 'treatment_09'} # does not make sense to plot for treatment in 2005 --> no data
for year, treatment in avg_dfs.items():
    avg_dfs[year] = ltw[['Land','Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', treatment, 'year']].groupby(['Land', treatment, 'year'], as_index = False).mean()
    avg_dfs[year].rename(columns = {f'{treatment}':'treatment'}, inplace = True)
    mean = ltw[['Land', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'year']].groupby(['Land', 'year'], as_index = False).mean()
    mean['treatment'] = 'Mean'
    avg_dfs[year] = pd.concat([avg_dfs[year], mean])
avg_dfs[2010].head(15)

,Land,treatment,year,Union,SPD,FDP,Linke,Grüne,Andere
0,BY,None,2008,47.209399,14.837163,6.872728,3.796851,7.568360,19.829492
1,BY,None,2013,52.398430,16.418284,2.604151,1.726068,6.754107,20.206445
2,BY,None,2018,41.749495,8.014001,4.047795,2.414772,13.196995,30.691564
3,BY,Within 15-30 km,2008,53.530268,15.815697,4.903882,4.241571,6.058939,15.589316
4,BY,Within 15-30 km,2013,51.926464,17.360894,2.552495,2.168486,8.077512,18.071850
5,BY,Within 15-30 km,2018,46.706655,8.408426,3.175199,2.462502,13.972875,25.465568
6,BY,Within 30-50 km,2008,52.632211,14.140743,6.847117,4.074680,6.009267,16.444342
7,BY,Within 30-50 km,2013,51.663829,15.818846,3.538632,2.234835,7.009037,19.880122
8,BY,Within 30-50 km,2018,44.527339,7.219252,3.701745,2.555107,12.640097,29.507158
9,HE,Direct Line,2009,40.290051,28.657739,14.049635,5.265776,8.061083,3.675716


In [69]:
states = {'BY': 'Bavaria', 'HE': 'Hesse', 'NW': 'North-Rhine Westphalia', 'NI': 'Lower Saxony'}
treatments = ['None', 'Direct Line', 'Within 0-15 km', 'Within 15-30 km', 'Within 30-50 km', 'Mean']
blacks = ['#000000', '#191919', '#323232', '#4c4c4c', '#666666', 'blue']
reds = ['#cc0000', '#d11919', '#d63232', '#db4c4c', '#e06666', 'blue']
yellows = ['#ffd075', '#ffc75c', '#ffbe42', '#ffb529', '#ffac0f', 'blue']
purples = ['#800080', '#8c198c', '#993299', '#a64ca6', '#b266b2', 'blue']
greens = ['#008000', '#198c19', '#329932', '#4ca64c' , '#66b266', 'blue']
greys = ['#808080', '#8c8c8c', '#999999', '#a6a6a6', '#b2b2b2', 'blue']
markers = ['o', 's', 'v', 'D', '^', 'h']
n = 10
for year, df in avg_dfs.items():
    for abb, state in states.items():
        # keep only obs. corresponding to state
        state_data = df[df['Land'] == abb]
        # get years in a list
        years = state_data.year.unique().astype(int).tolist()
        # initialize figure 
        fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 10), dpi=600)
        # set x labels
        plt.setp(axes, xticks=years)
        # set legend font size
        plt.rc('legend',fontsize=6)
        # title for entire figure
        fig.suptitle(f'Fig. E{n} \n {state} State Parliament Elections \n Average Vote Shares by treatment status in {year}', fontsize=16)

        for i, treatment in enumerate(treatments):
            plot_data = state_data[state_data['treatment'] == treatment]
            if plot_data.empty:
                continue
            # set ls
            if treatment == 'Mean':
                ls = '--'
            else:
                ls = '-'
            # edit subplots
            axes[0, 0].set_title('Union', fontsize=12)
            axes[0, 0].set_ylabel('% Votes', fontsize=10)
            axes[0, 0].plot(plot_data['year'], plot_data['Union'], c=blacks[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
            axes[0, 0].legend(loc='lower left')
            axes[0, 0].axvline(x=year, c='red', ls='--')

            axes[0, 1].set_title('SPD', fontsize=12)
            axes[0, 1].set_ylabel('% Votes', fontsize=10)
            axes[0, 1].plot(plot_data['year'], plot_data['SPD'], c=reds[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
            axes[0, 1].legend(loc='lower left')
            axes[0, 1].axvline(x=year, c='red', ls='--')

            axes[1, 0].set_title('FDP', fontsize=12)
            axes[1, 0].set_ylabel('% Votes', fontsize=10)
            axes[1, 0].plot(plot_data['year'], plot_data['FDP'], c=yellows[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
            axes[1, 0].legend(loc='upper left')
            axes[1, 0].axvline(x=year, c='red', ls='--')

            axes[1, 1].set_title('Linke', fontsize=12)
            axes[1, 1].set_ylabel('% Votes', fontsize=10)
            axes[1, 1].plot(plot_data['year'], plot_data['Linke'], c=purples[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
            axes[1, 1].legend(loc='upper left')
            axes[1, 1].axvline(x=year, c='red', ls='--')

            axes[2, 0].set_title('Grüne', fontsize=12)
            axes[2, 0].set_ylabel('% Votes', fontsize=10)
            axes[2, 0].plot(plot_data['year'], plot_data['Grüne'], c=greens[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
            axes[2, 0].legend(loc='upper left')
            axes[2, 0].axvline(x=year, c='red', ls='--')

            axes[2, 1].set_title('Andere', fontsize=12)
            axes[2, 1].set_ylabel('% Votes', fontsize=10)
            axes[2, 1].plot(plot_data['year'], plot_data['Andere'], c=greys[i], marker = markers[i], label=treatment, lw=0.8, ms=4, ls=ls)
            axes[2, 1].legend(loc='upper left')
            axes[2, 1].axvline(x=year, c='red', ls='--')

        # Save figure
        fig.savefig(f'{path}/figures/py/E{n}_ltw_{abb}_{year}_vote_shares.png')
        plt.close()
        figs_tex[n] = (fig_tex % (f'E{n}', f'E{n}_ltw_{abb}_{year}_vote_shares.png', '{state} State Parliament Elections \n Average Vote Shares by treatment status in {year}', f'E{n}'))
        n = n + 1 

Fig. E18-E21: Maps with the partys who won the elections and treatment status

In [70]:
# Load shapefiles
import geopandas as gpd
municipalities = gpd.read_file(f'{path}/data/geodata/municipalities.shp')
states = gpd.read_file(f'{path}/data/geodata/VG250_LAN.shp')
powerlines = gpd.read_file(f'{path}/data/geodata/all_powerlines.shp')
powerlines = powerlines.set_geometry('geometry')
powerlines['year'] = powerlines['year'].astype(int)
# set year to treatment year
powerlines['year'] = np.where(powerlines['year'] == 2013, 2014, powerlines['year'])
powerlines['year'] = np.where(powerlines['year'] == 2009, 2010, powerlines['year'])

In [71]:
# Replace State IDs with Abbreviations 
abb = {'01': 'SH', '02': 'HH', '03': 'NI', '04': 'HB', '05': 'NW', '06': 'HE', '07': 'RP', '08': 'BW', \
         '09': 'BY', '10': 'SL', '11': 'BE', '12': 'BB', '13': 'MV', '14': 'SN', '15': 'ST', '16': 'TH'}
for key, value in abb.items():
    municipalities['SN_L'] = np.where(municipalities['SN_L'] == key, value, municipalities['SN_L'])
    states['SN_L'] = np.where(states['SN_L'] == key, value, states['SN_L'])

In [72]:
# Prepare df with election winners
ltw_win = ltw[['AGS', 'Land', 'year', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']]
ltw_win['winner'] = ltw_win[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']].idxmax(axis = 1)
ltw_win = ltw_win[['AGS', 'Land', 'year', 'winner']]
ltw_win['winner'].describe()

# add color column to df
colors = {'Union': '#000000', 'SPD': '#cc0000', 'FDP': '#ffff00', 'Grüne': '#008000', 'Linke': '#800080', 'Andere': '#808080'}
ltw_win['color'] = 0
for party, color in colors.items():
    ltw_win['color'] = np.where(ltw_win['winner'] == party, color, ltw_win['color'])

# Merge winners and municipalities
ltw_win_plot = pd.merge(ltw_win, municipalities, on='AGS')
ltw_win_plot = ltw_win_plot.set_geometry('geometry')

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_85109/2492951917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ltw_win['winner'] = ltw_win[['Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']].idxmax(axis = 1)


In [73]:
# Prepare df with treatment status
# First subset ltw and rename columns
#ltw_treat = ltw[['AGS', 'year', 'Land', 'treatment_05', 'ever_treated_2010', 'ever_treated']]
#ltw_treat.rename(columns = {'treatment_05':'t2005', 'ever_treated_2010':'t2010', 'ever_treated':'t2014'}, inplace = True)
# Merge win and municipalities
ltw_treatment_plot = pd.merge(ltw, municipalities, on='AGS')
ltw_treatment_plot = ltw_treatment_plot.set_geometry('geometry')

In [74]:
from geopandas.tools import overlay
from matplotlib.patches import Circle, Wedge, Polygon
from matplotlib.collections import PatchCollection
abb = {'BY': 'Bavaria', 'HE': 'Hesse', 'NW': 'North-Rhine Westphalia', 'NI': 'Lower Saxony'}
legend_dict = {2005: 'DENA 2005', 2010: 'EnLAG 2009', 2014: 'BBPlG 2013'}
n = 18
for abb, state in abb.items():
    # keep only obs. corresponding to state
    state_winners = ltw_win_plot[ltw_win_plot['Land'] == abb]
    state_treated = ltw_treatment_plot[ltw_treatment_plot['Land'] == abb]
    state_treated['year'] = state_treated['year'].astype(str)
    states_plot = states[states['SN_L'] == abb]
    # keep only parts of powerlines that intersect state
    powerlines_state = powerlines.overlay(states_plot, how='intersection')
    # get years in a list
    years = state_winners.year.unique().astype(int).tolist()
    # initialize figure 
    fig = plt.figure(layout="constrained", figsize=(10, 10), dpi=600)
    subfigs = fig.subfigures(2, 1, height_ratios=[6., 1.])
    axs0 = subfigs[0].subplots(3, 2)
    axs1 = subfigs[1].subplots(1, 5)
    for ax1 in axs1:
        ax1.set_aspect('equal')
        ax1.set_axis_off()
    # title for entire figure
    fig.suptitle(f'Fig. E{n} \n {state} election winners and treated municipalities', fontsize=16)
    for i, year in enumerate(years):
        # subset df for year
        plot_winners = state_winners[state_winners['year'] == year]
        # subset powerlines
        plot_pl = powerlines_state[powerlines_state['year'] <= year]
        # draw subfigs
        # Treated municipalities in first column
        axs0[i, 0].set_aspect('equal')
        axs0[i, 0].set_adjustable('datalim', share=True)
        axs0[i, 0].set_axis_off()

        if i == 0:
            axs0[i, 0].set_title(f'\nTreated municipalities\n{year}')
        else: 
            axs0[i, 0].set_title(f'{year}')
        if year >= 2005 and year < 2010:
            state_treated.plot(ax=axs0[i, 0], column='t2005', cmap='viridis', zorder=1)
        if year >= 2010 and year < 2014:
            state_treated.plot(ax=axs0[i, 0], column='et2010', cmap='viridis', zorder=1)
        if year >= 2014:
            state_treated.plot(ax=axs0[i, 0], column='et2014', cmap='viridis', zorder=1)
        # Powerlines 
        plot_pl.plot(ax=axs0[i, 0], color=plot_pl['color'], lw=2, zorder=2)
        # Election winners in second column
        axs0[i, 1].set_aspect('equal')
        axs0[i, 1].set_adjustable('datalim', share=True)
        axs0[i, 1].set_axis_off()
        if i == 0:
            axs0[i, 1].set_title(f'\nElection winners\n{year}')
        else: 
            axs0[i, 1].set_title(f'{year}')
        plot_winners.plot(ax=axs0[i, 1], color=plot_winners['color'], zorder=1)
        # Powerlines 
        plot_pl.plot(ax=axs0[i, 1], color=plot_pl['color'], lw=2, zorder=2)
    # Add legends    
    # add handles for the legends
    p_handles = [
        Circle([0], [0], color='#000000'),
        Circle([0], [0], color='#cc0000'),
        Circle([0], [0], color='#ffff00'),
        Circle([0], [0], color='#800080'),
        Circle([0], [0], color='#008000'),
        Circle([0], [0], color='#808080')
    ]
    pl_handles = [
        Circle([0], [0], color='#1ED4CE'),
        Circle([0], [0], color='#B8FDE7'),
        Circle([0], [0], color='#1B5EB6')
    ]
    t_handles = [
        Circle([0], [0], color='#440154'),
        Circle([0], [0], color='#31688e'),
        Circle([0], [0], color='#35b779'),
        Circle([0], [0], color='#fde725')
    ]


    # define labels for the legend
    p_labels = ['CDU', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
    pl_labels = ['DENA 2005', 'EnLAG 2009', 'BBPlG 2013']
    t_labels = ['Direct Line', 'Within 0-15 km', 'Within 15-30 km', 'Within 30-50 km']
    # plot legends
    axs1[1].legend(handles=t_handles, labels=t_labels, loc='upper left', title='Treatments', ncol=2)
    axs1[2].legend(handles=pl_handles, labels=pl_labels, loc='upper center', title='Powerlines', ncol=1)
    axs1[3].legend(handles=p_handles, labels=p_labels, loc='upper right', title='Parties', ncol=2)
    plt.savefig(f'{path}/figures/py/E{n}_{abb}_win_and_treat.png', bbox_inches="tight", pad_inches=0)
    plt.close()
    figs_tex[n] = (fig_tex % (f'E{n}', f'E{n}_{abb}_win_and_treat.png', '{state} election winners and treated municipalities', f'E{n}'))
    n = n + 1

/opt/homebrew/Caskroom/miniconda/base/envs/vscode/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniconda/base/envs/vscode/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniconda/base/envs/vscode/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning:

First, import needed modules

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D

Initalize variables

In [76]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [77]:
controls = pd.read_csv(f'{path}/data/controls.csv', encoding = 'UTF-8-SIG', sep=',', converters={'AGS': str})
treatment = pd.read_csv(f'{path}/data/treatment.csv', converters = {'AGS': str})

Merge df

In [78]:
control_treat = pd.merge(controls, treatment, on='AGS', how='outer', indicator=True)
control_treat['_merge'].value_counts()

both          336665
left_only       6382
right_only         6
Name: _merge, dtype: int64

Drop early and late years

In [79]:
control_treat = control_treat[control_treat['year'] > 1993]
control_treat = control_treat[control_treat['year'] < 2018]

Fig E22: Controls trends by ever-treated status

In [80]:
avg_ever_treated = control_treat[['pop_density', 'unemployed', 'avg_age', 'female', 'ever_treated', 'year']].groupby(['ever_treated', 'year'], as_index = False).agg([np.mean, np.std, 'count'])
avg_ever_treated = avg_ever_treated.reset_index()
avg_ever_treated.rename(columns = {'ever_treated':'treatment'}, inplace = True)
mean = control_treat[['pop_density', 'unemployed', 'avg_age', 'female', 'year']].groupby(['year'], as_index = False).agg([np.mean, np.std, 'count'])
mean = mean.reset_index()
mean['treatment'] = 'Mean'
avg_ever_treated = pd.concat([avg_ever_treated, mean])
avg_ever_treated[avg_ever_treated['treatment'] == 'Mean'].head()

treatment    year pop_density                    unemployed            \
                           mean         std  count       mean       std   
0      Mean  1994.0  177.983198  273.726801  11235        NaN       NaN   
1      Mean  1995.0  179.495683  273.649804  11235        NaN       NaN   
2      Mean  1996.0  180.660328  273.530082  11235        NaN       NaN   
3      Mean  1997.0  181.543290  272.758212  11235        NaN       NaN   
4      Mean  1998.0  182.325160  272.530649  11235   4.562052  2.441889   

         avg_age               female                   
   count    mean std count       mean       std  count  
0      0     NaN NaN     0        NaN       NaN      0  
1      0     NaN NaN     0  50.554116  1.152348  10799  
2      0     NaN NaN     0  50.516251  1.035296  10799  
3      0     NaN NaN     0  50.497382  1.019976  10799  
4  10799     NaN NaN     0  50.473580  1.016162  10799

In [81]:
treatments = ['None', 'Direct Line', 'Within 15km', 'Within 30km', 'Within 50km', 'Mean']
blacks = ['#cc0000', '#d11919', '#d63232', '#db4c4c', '#e06666', 'blue']
markers = ['o', 's', 'v', 'D', '^', 'h']

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10), dpi=300, sharex=False)
# set legend font size
plt.rc('legend',fontsize=6)
# title for entire figure
fig.suptitle(f'Fig. E22 \n Averages values of control variables \n by ever treated status \n', fontsize=16)
#for ax in axes:
    #ax.set_xticks([1994,1998,2002,2005,2009,2013,2017])
for i, treatment in enumerate(treatments):
    plot_data = avg_ever_treated[avg_ever_treated['treatment'] == treatment]
    # edit subplots
    axes[0, 0].set_title('Population Density', fontsize=12)
    axes[0, 0].set_ylabel('Population Density', fontsize=10)
    axes[0, 0].errorbar(plot_data['year'], plot_data['pop_density', 'mean'], c=blacks[i], marker = markers[i], label=treatment, lw=0.8, ms=4, capsize=4)
    axes[0, 0].legend(loc='upper left')
    axes[0, 0].set_ylim(140, 320)
    axes[0, 0].set_xticks([1994,1998,2002,2005,2009,2013,2017])
    axes[0, 0].tick_params(labelsize = 8)

    axes[0, 1].set_title('Share of Unemployed', fontsize=12)
    axes[0, 1].set_ylabel('% Unemployed', fontsize=10)
    axes[0, 1].errorbar(plot_data['year'], plot_data['unemployed', 'mean'], c=blacks[i], marker = markers[i], label=treatment, lw=0.8, ms=4, capsize=4)
    axes[0, 1].legend(loc='upper left')
    axes[0, 1].set_ylim(2, 6)
    axes[0, 1].set_xlim(1997, 2018)
    axes[0, 1].set_xticks([1998,2002,2005,2009,2013,2017])
    axes[0, 1].tick_params(labelsize = 8)

    axes[1, 0].set_title('Average Age', fontsize=12)
    axes[1, 0].set_ylabel('Average Age', fontsize=10)
    axes[1, 0].errorbar(plot_data['year'], plot_data['avg_age', 'mean'], c=blacks[i], marker = markers[i], label=treatment, lw=0.8, ms=4, capsize=4)
    axes[1, 0].legend(loc='upper left')
    axes[1, 0].set_ylim(40, 46)
    axes[1, 0].set_xlim(2000, 2018)
    axes[1, 0].set_xticks([2002,2005,2009,2013,2017])
    axes[1, 0].tick_params(labelsize = 8)

    axes[1, 1].set_title('Share of Females', fontsize=12)
    axes[1, 1].set_ylabel('% Female', fontsize=10)
    axes[1, 1].errorbar(plot_data['year'], plot_data['female', 'mean'], c=blacks[i], marker = markers[i], label=treatment, lw=0.8, ms=4, capsize=4)
    axes[1, 1].legend(loc='upper left')
    axes[1, 1].set_ylim(49.9, 51.1)
    axes[1, 1].set_xlim(1994, 2018)
    axes[1, 1].set_xticks([1994,1998,2002,2005,2009,2013,2017])
    axes[1, 1].tick_params(labelsize = 8)

# Save figure
fig.savefig(f'{path}/figures/py/E22_ever_treated_controls.png')
figs_tex[n] = (fig_tex % (f'E{22}', f'E22_ever_treated_controls.png', 'Averages values of control variables by ever treated status', f'E{22}'))
plt.close()

Fig E23-E25: Controls trends by treatment year

In [82]:
avg_dfs = {2005: 'treatment_05', 2010: 'treatment_09', 2014: 'treatment_13'}
for year, treatment in avg_dfs.items():
    avg_dfs[year] = control_treat[['pop_density', 'unemployed', 'avg_age', 'female', treatment, 'year']].groupby([treatment, 'year'], as_index = False).agg([np.mean, np.std, 'count'])
    avg_dfs[year] = avg_dfs[year].reset_index()
    avg_dfs[year].rename(columns = {treatment:'treatment'}, inplace = True)
    mean = control_treat[['pop_density', 'unemployed', 'avg_age', 'female', 'year']].groupby(['year'], as_index = False).agg([np.mean, np.std, 'count'])
    mean = mean.reset_index()
    mean['treatment'] = 'Mean'
    avg_dfs[year] = pd.concat([avg_dfs[year], mean])
avg_dfs[2005][avg_dfs[2005]['treatment'] == 'Mean'].head(40)

treatment    year pop_density                    unemployed            \
                            mean         std  count       mean       std   
0       Mean  1994.0  177.983198  273.726801  11235        NaN       NaN   
1       Mean  1995.0  179.495683  273.649804  11235        NaN       NaN   
2       Mean  1996.0  180.660328  273.530082  11235        NaN       NaN   
3       Mean  1997.0  181.543290  272.758212  11235        NaN       NaN   
4       Mean  1998.0  182.325160  272.530649  11235   4.562052  2.441889   
5       Mean  1999.0  183.131933  271.172853  11235   4.352015  2.440473   
6       Mean  2000.0  183.743671  271.892317  11235   4.207775  2.698692   
7       Mean  2001.0  184.315102  272.334939  11235   4.273641  2.866527   
8       Mean  2002.0  184.603404  272.895422  11235   4.490443  2.878124   
9       Mean  2003.0  184.637231  272.734872  11235   4.862838  3.015907   
10      Mean  2004.0  184.722517  273.028394  11235   4.865108  3.070327   
11      Mean  2005.0  184.548303  273.837692  11235   5.199646  2.924148   
12      Mean  2006.0  184.065419  274.036161  11235   4.723538  2.774153   
13      Mean  2007.0  183.693201  274.342249  11235   3.905358  2.511615   
14      Mean  2008.0  183.034397  274.356080  11235   3.308165  2.204572   
15      Mean  2009.0  182.511059  274.601614  11235   3.459419  1.953894   
16      Mean  2010.0  182.208634  275.192534  11235   3.213549  1.796892   
17      Mean  2011.0  181.976198  276.325922  11235   2.937928  1.817268   
18      Mean  2012.0  180.023644  272.272375  11235   2.827625  1.708779   
19      Mean  2013.0  180.175584  273.909646  11235   2.873463  1.629631   
20      Mean  2014.0  180.975135  276.112810  11235   2.744451  1.499770   
21      Mean  2015.0  182.662125  279.901456  11235   2.575198  1.381288   
22      Mean  2016.0  183.287438  281.820142  11235   2.452648  1.249592   
23      Mean  2017.0  183.730276  283.218212  11235   2.253809  1.092626   

             avg_age                      female                   
    count       mean       std  count       mean       std  count  
0       0        NaN       NaN      0        NaN       NaN      0  
1       0        NaN       NaN      0  50.554116  1.152348  10799  
2       0        NaN       NaN      0  50.516251  1.035296  10799  
3       0        NaN       NaN      0  50.497382  1.019976  10799  
4   10799        NaN       NaN      0  50.473580  1.016162  10799  
5   10799        NaN       NaN      0  50.456501  1.005093  10799  
6   10799        NaN       NaN      0  50.446845  0.997520  10799  
7   10799  40.241344  1.774452  10799  50.420910  0.999051  10799  
8   10799  40.555035  1.801721  10799  50.393882  0.993415  10799  
9   10798  40.897454  1.816544  10798  50.396477  0.989580  10798  
10  10798  41.270966  1.847474  10798  50.383901  0.989946  10798  
11  10798  41.655742  1.881154  10798  50.363922  0.995950  10798  
12  10798  42.068166  1.906464  10798  50.333968  0.997188  10798  
13  10798  42.439602  1.932873  10798  50.311415  1.007818  10798  
14  10798  42.810252  1.961271  10798  50.291731  1.013217  10798  
15  10798  43.186573  1.985881  10798  50.268841  1.019449  10798  
16  10798  43.524096  2.005315  10798  50.240130  1.016324  10798  
17  10797  43.844815  2.026106  10797  50.406841  0.926695  10797  
18  10797  44.156630  2.035843  10797  50.361241  0.920615  10797  
19  10797  44.431115  2.047883  10797  50.307307  0.927477  10797  
20  10797  44.671898  2.066090  10797  50.248659  0.926714  10797  
21  10797  44.752964  2.098749  10797  50.060685  1.008222  10797  
22  10797  44.912226  2.364855  10797  50.070324  0.924571  10797  
23  10797  45.108010  2.129146  10797  50.071169  0.913835  10797

In [83]:
treatments = ['None', 'Direct Line', 'Within 15km', 'Within 30km', 'Within 50km', 'Mean']
blacks = ['#cc0000', '#d11919', '#d63232', '#db4c4c', '#e06666', 'blue']
markers = ['o', 's', 'v', 'D', '^', 'h']
n = 23
for year, df in avg_dfs.items():
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10), dpi=300, sharex=False)
    # set legend font size
    plt.rc('legend',fontsize=6)
    # title for entire figure
    fig.suptitle(f'Fig. E{n} \n Averages values of control variables \n by treatment status in {year}', fontsize=16)
    for i, treatment in enumerate(treatments):
        plot_data = df[df['treatment'] == treatment]
        # edit subplots
        axes[0, 0].set_title('Population Density', fontsize=12)
        axes[0, 0].set_ylabel('Population Density', fontsize=10)
        axes[0, 0].errorbar(plot_data['year'], plot_data['pop_density', 'mean'], c=blacks[i], marker = markers[i], label=treatment, lw=0.8, ms=4, capsize=4)
        axes[0, 0].legend(loc='upper left')
        axes[0, 0].axvline(x=year, c='red', linestyle='--')
        axes[0, 0].set_ylim(140, 320)
        axes[0, 0].set_xlim(1994, 2018)
        axes[0, 0].set_xticks([1994,1998,2002,2005,2009,2013,2017])
        axes[0, 0].tick_params(labelsize = 8)

        axes[0, 1].set_title('Share of Unemployed', fontsize=12)
        axes[0, 1].set_ylabel('% Unemployed', fontsize=10)
        axes[0, 1].errorbar(plot_data['year'], plot_data['unemployed', 'mean'], c=blacks[i], marker = markers[i], label=treatment, lw=0.8, ms=4, capsize=4)
        axes[0, 1].legend(loc='upper left')
        axes[0, 1].axvline(x=year, c='red', linestyle='--')
        axes[0, 1].set_ylim(2, 6)
        axes[0, 1].set_xlim(1997, 2018)
        axes[0, 1].set_xticks([1998,2002,2005,2009,2013,2017])
        axes[0, 1].tick_params(labelsize = 8)

        axes[1, 0].set_title('Average Age', fontsize=12)
        axes[1, 0].set_ylabel('Average Age', fontsize=10)
        axes[1, 0].errorbar(plot_data['year'], plot_data['avg_age', 'mean'], c=blacks[i], marker = markers[i], label=treatment, lw=0.8, ms=4, capsize=4)
        axes[1, 0].legend(loc='upper left')
        axes[1, 0].axvline(x=year, c='red', linestyle='--')
        axes[1, 0].set_ylim(40, 46)
        axes[1, 0].set_xlim(2000, 2018)
        axes[1, 0].set_xticks([2002,2005,2009,2013,2017])
        axes[1, 0].tick_params(labelsize = 8)

        axes[1, 1].set_title('Share of Females', fontsize=12)
        axes[1, 1].set_ylabel('% Female', fontsize=10)
        axes[1, 1].errorbar(plot_data['year'], plot_data['female', 'mean'], c=blacks[i], marker = markers[i], label=treatment, lw=0.8, ms=4, capsize=4)
        axes[1, 1].legend(loc='upper left')
        axes[1, 1].axvline(x=year, c='red', linestyle='--')
        axes[1, 1].set_ylim(49.9, 51.1)
        axes[1, 1].set_xlim(1994, 2018)
        axes[1, 1].set_xticks([1994,1998,2002,2005,2009,2013,2017])
        axes[1, 1].tick_params(labelsize = 8)

    # Save figure
    fig.savefig(f'{path}/figures/py/E{n}_{year}_controls.png')
    plt.close()
    figs_tex[n] = (fig_tex % (f'E{n}', f'E{n}_{year}_controls.png', 'Averages values of control variables by treatment status in {year}', f'E{n}'))
    n = n + 1

Fig E30-E33: Choropleth maps of controls by year

In [84]:
municipalities = gpd.read_file(f'{path}/data/geodata/municipalities.shp')
municipalities = municipalities.set_geometry('geometry')
states = gpd.read_file(f'{path}/data/geodata/VG250_LAN.shp')
powerlines = gpd.read_file(f'{path}/data/geodata/all_powerlines.shp')
powerlines = powerlines.set_geometry('geometry')
powerlines['year'] = powerlines['year'].astype(int)
# set year to treatment year
powerlines['year'] = np.where(powerlines['year'] == 2013, 2014, powerlines['year'])
powerlines['year'] = np.where(powerlines['year'] == 2009, 2010, powerlines['year'])

In [85]:
controls_muns = pd.merge(controls, municipalities, on='AGS')
controls_muns.shape

(336665, 15)

In [86]:
#Drop not needed years for plot
years = [2005, 2010, 2014]
controls_muns = controls_muns[controls_muns['year'].isin(years)]
controls_muns.shape

(33757, 15)

In [87]:
legend_dict = {2005: 'DENA 2005', 2010: 'EnLAG 2009', 2014: 'BBPlG 2013'}
control_vars = {'pop_density': 'Population Density', 'unemployed': 'Share of unemployed', 'avg_age': 'Average Age', 'female': 'Share of Females'}
labels = control_vars = {'pop_density': 'Inhabitants per square kilometer', 'unemployed': 'Share of unemployed in percent of the population', 'avg_age': 'Average Age in years', 'female': 'Share of Females in percent of the population'}
n = 26
for var, title in control_vars.items():
    # initialize figure 
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 7), dpi=300, constrained_layout=True)
    # title for entire figure
    fig.suptitle(f'Fig. E{n} \n {title}', fontsize=16)
    for i, (year, desc) in enumerate(legend_dict.items()):
        # subset powerlines
        plot_pl = powerlines[powerlines['year'] == year]
        # subset controls plot
        controls_plot = controls_muns[controls_muns['year'] == year]
        controls_plot = controls_plot.set_geometry('geometry')
        # get min and max values
        min = controls_muns.min(axis=0, numeric_only=True)[var]
        max = controls_muns.max(axis=0, numeric_only=True)[var]
        cmap = 'viridis'
        # draw subfig
        ax[i].set_aspect('equal')
        ax[i].set_adjustable('datalim', share=True)
        ax[i].set_axis_off()
        ax[i].set_title(f'\n {desc}')
        controls_plot.plot(column=var, ax=ax[i], vmin=min, vmax=max, cmap=cmap, legend_kwds={'orientation': "horizontal"}, zorder=1)
        states.boundary.plot(ax=ax[i], color='white', lw = 0.1, zorder=3)
        plot_pl.plot(ax=ax[i], color='red', lw=2, zorder=3)
    # define a mappable based on which the colorbar will be drawn
    mappable = cm.ScalarMappable(norm=mcolors.Normalize(min, max), cmap=cmap)
    # define position and extent of colorbar
    cb_ax = fig.add_axes([0.1, 0.1, 0.8, 0.05])
    # draw colorbar
    cbar = fig.colorbar(mappable, cax=cb_ax, orientation='horizontal', label = labels[var])
    plt.savefig(f'{path}/figures/py/E{n}_map_{var}.png', bbox_inches="tight", pad_inches=0)
    plt.close()
    figs_tex[n] = (fig_tex % (f'E{n}', f'E{n}_map_{var}.png', '{title}', f'E{n}'))
    print(var)
    n = n + 1

pop_density
unemployed
avg_age
female


In [88]:
for value in figs_tex.values():
    print(value)

(\renewcommand{\thefigure}{E1} \begin{figure} \centering \includegraphics[width=0.9\textwidth]{Figures/E1_all_treatments.png} \captionlistentry{Municipalities affected by Powerline Projects} \label{fig:E1} \end{figure} \clearpage)
(\renewcommand{\thefigure}{E2} \begin{figure} \centering \includegraphics[width=0.9\textwidth]{Figures/E2_all_treatments_sum.png} \captionlistentry{Treated municipalities by year} \label{fig:E2} \end{figure} \clearpage)
(\renewcommand{\thefigure}{E4} \begin{figure} \centering \includegraphics[width=0.9\textwidth]{Figures/E4_gtrends.png} \captionlistentry{Google Search interest for the term "Stromtrasse"} \label{fig:E4} \end{figure} \clearpage)
(\renewcommand{\thefigure}{E3} \begin{figure} \centering \includegraphics[width=0.9\textwidth]{Figures/E3_btw_avg_vote_shares.png} \captionlistentry{Average First and Second Vote Shares in Federal Elections} \label{fig:E3} \end{figure} \clearpage)
(\renewcommand{\thefigure}{E5} \begin{figure} \centering \includegraphics